<a href="https://colab.research.google.com/github/lukaszplust/Projects/blob/main/proba_1_sbd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TEST

WYNIK:

Record 1 as ints: [3, 3, 5, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Record 2 as ints: [4, 4, 2, 6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Record 1 loaded: Set [5, 3, 2]
Record 2 loaded: Set [6, 4, 2, 1]
Record 1 < Record 2: True
Record 2 < Record 1: False

WYJAŚNIENIE:

Record 1 < Record 2 zwraca True, ponieważ największy element w Record 2 (6) jest większy niż największy element w Record 1 (5).
Record 2 < Record 1 zwraca False, ponieważ największy element w Record 1 (5) nie jest większy niż największy element w Record 2 (6).

In [43]:
# Tworzenie rekordów
#record1 = Record([3, 5, 2])
#record2 = Record([4, 2, 6, 1])

# Konwersja rekordów do listy liczb całkowitych
#record1_ints = record1.save_to_ints()
#record2_ints = record2.save_to_ints()
#print("Record 1 as ints:", record1_ints)
#print("Record 2 as ints:", record2_ints)

# Tworzenie rekordów z listy liczb całkowitych
#record1_loaded = Record.load_from_ints(record1_ints)
#record2_loaded = Record.load_from_ints(record2_ints)
#print("Record 1 loaded:", record1_loaded)
#print("Record 2 loaded:", record2_loaded)

# Porównanie rekordów
#print("Record 1 < Record 2:", record1 < record2)
#print("Record 2 < Record 1:", record2 < record1)

In [49]:
# BUFFER_SIZE - określa liczbę rekordów, które będą przechowywane w buforze w jednym momencie.
# Buforowanie poprawia wydajność operacji wejścia-wyjścia, minimalizując liczbę operacji odczytu i zapisu na dysk poprzez grupowanie ich w większe bloki.
BUFFER_SIZE = 32


# SET_BYTES_SIZE - określa rozmiar w bajtach pojedynczego rekordu bez dodatkowych znaków. Zakłada się, że rekord zawiera dane o stałej długości (4 liczy * 2cyfry +3 spacje)
SET_BYTES_SIZE = 15

# Ta stała określa rozmiar w bajtach pojedynczego rekordu, włączając dodatkowy znak (nowa linia lub znak końca rekordu).
# Zazwyczaj jest to znak '\n' (nowa linia), który oddziela rekordy w pliku tekstowym
RECORD_BYTES_SIZE = SET_BYTES_SIZE + 1

# Ta stała określa całkowity rozmiar bufora w bajtach.
# Jest to iloczyn liczby rekordów w buforze (BUFFER_SIZE) i rozmiaru jednego rekordu w bajtach (RECORD_BYTES_SIZE).
# Bufor o takim rozmiarze będzie używany do operacji odczytu i zapisu blokowego, co zwiększa efektywność przez minimalizację liczby operacji I/O
BYTES_BUFFER_SIZE = BUFFER_SIZE * RECORD_BYTES_SIZE


Należy zaimplementować symulację odczytu oraz zapisu blokowego jako oddzielną
warstwę logiki programu. Warstwa ta powinna udostępniać warstwie sorowania co najmniej 2
operacje: odczytu oraz zapisu pojedynczego rekordu.

In [45]:
import os
class ReadBuffer:

  def __init__(self, file_path):
    #pdb.set_trace()
    # pozycja odczytu w buforze
    self.read_pos = 0
    # rozmiar bufora, ustawiony na BUFFER_SIZE
    self.size = BUFFER_SIZE
    # liczba rekordów obecnie załadowanych do bufora
    self.loaded_size = 0
    # lista przechowująca rekordy załadowane do bufora
    self.buffer = []
    # ścieżka do pliku, z którego będą odczytywane rekordy
    self.file_path = file_path
    # pozycja odczytu w pliku
    self.file_pos = 0
    # całkowity rozmiar pliku (w bajtach)
    self.file_size = os.path.getsize(file_path)
    # licznik operacji odczytu z dysku
    self.disk_reads_count = 0
    # inicjalnie ładuje pierwszą porcję danych do bufora
    self.load_next()


  # ODCZYT REKORDÓW: Metoda read_next zwraca następny rekord
  def read_next(self):
    #pdb.set_trace()
    # sprawdzam, czy są jeszcze rekordy do odczytu
    if not self.has_more():
        return None

    # jeśli są jakieś rekordy do odczytu to zwracam następny rekord z bufora
    res_record = self.buffer[self.read_pos]
    self.read_pos += 1

    # jeśli pozycja odczytu osiągnie rozmiar bufora
    if self.read_pos == self.size:

        # ładuje kolejną porcję danych do bufora (self.load_next())
        self.load_next()

        # resetuje self.read_pos
        self.read_pos = 0


    return res_record


  # Muszę sprawdzić, czy są jeszcze dane do odczytu w pliku lub w buforze.
  def has_more(self):
        return (self.file_pos < self.file_size or self.read_pos < self.loaded_size)

  # PODGLĄD NASTĘPNEGO REKORDU: Metoda peek pozwala na podgląd następnego rekordu bez przesuwania wskaźnika odczytu
  # Zwracam następny rekord bez zmiany pozycji odczytu. Jeśli bufor jest pusty, zwracam None.
  def peek(self):

    # pozycja odczytu w pliku = liczba rekordów obecnie załadowanych do bufora
    if self.read_pos == self.loaded_size:
        return None
    result = self.buffer[self.read_pos]
    return result

  # ŁADOWANIE KOLEJNEJ PORCJI DANYCH: Metoda load_next ładuje nową porcję danych z pliku do bufora.
  def load_next(self):
        #pdb.set_trace()
        self.buffer = []
        # buffering=0 disables buffering, it is desired, because buffering is implemented here, in code

        # otwieram plik w trybie binarnym bez buforowania (buffering=0)
        file = open(self.file_path, "rb", buffering=0)

        # przesuwam wskaźnik odczytu pliku do self.file_pos
        file.seek(self.file_pos)

        # odczytuje maksymalnie BYTES_BUFFER_SIZE bajtów lub pozostałe bajty do końca pliku (self.file_size - self.file_pos)
        bytes_to_read = min(BYTES_BUFFER_SIZE,self.file_size - self.file_pos)

        temp_buffer = file.read(bytes_to_read)

        # sprawdzam, czy liczba odczytanych bajtów jest wielokrotnością rozmiaru rekordu (RECORD_BYTES_SIZE)
        if len(temp_buffer) % RECORD_BYTES_SIZE != 0:
            raise Exception("Read bytes are not multiply of record size")

        # aktualizuje pozycję odczytu pliku (self.file_pos)
        self.file_pos += bytes_to_read

        # obliczam liczbę rekordów załadowanych do bufora (self.loaded_size)
        self.loaded_size = bytes_to_read / RECORD_BYTES_SIZE

        # Konwertuje odczytane bajty na listę rekordów
        temp_ints = list(temp_buffer)

        # dodaje liste rekordów do bufora (self.buffer)
        for i in range(len(temp_buffer) // RECORD_BYTES_SIZE):
            record_ints = temp_ints[
                          RECORD_BYTES_SIZE * i:RECORD_BYTES_SIZE * (i + 1)
                          ]
            self.buffer.append(Record.load_from_ints(record_ints))

        # zamykam plik i zwiększam licznik operacji odczytu z dysku (self.disk_reads_count)
        file.close()
        self.disk_reads_count += 1



        # ITERACJA: Klasa jest iterowalna dzięki metodom __iter__ i __next__
        # Metody __iter__ i __next__ są częścią protokołu iteracji w Pythonie. Dzięki tym metodom można sprawić, że obiekt będzie iterowalny,
        # co oznacza, że można go używać w konstrukcjach takich jak pętle for, list comprehensions,
        # funkcje map, filter oraz inne operacje, które wymagają iterowalnych obiektów

        # INICJALIZACJA BUFORA: Konstruktor __init__ otwiera plik i ładuje pierwszą porcję danych do bufora
        # __iter__: Zwraca iterator (sam obiekt ReadBuffer)
        def __iter__(self):
          return self

        # __next__: Zwraca następny rekord z bufora. Jeśli nie ma więcej rekordów, podnosi wyjątek StopIteration
        def __next__(self):
          next_record = self.read_next()
          if next_record is None:
            raise StopIteration
          return next_record

In [55]:
# Klasa WriteBuffer implementuje mechanizm buforowania zapisu danych do pliku,
# co może znacząco poprawić wydajność poprzez grupowanie operacji zapisu i minimalizację operacji na dysku.

#Jej działanie jest zoptymalizowane pod kątem pracy z dużą ilością danych, pozwalając na kontrolowanie i
# monitorowanie liczby zapisów na dysk oraz zachowanie kolejności zapisów (runów).

class WriteBuffer:

    # file_path: Ścieżka do pliku, do którego będą zapisywane dane
    # append_mode: Parametr opcjonalny, domyślnie ustawiony na False.
    # Jeśli ustawiony na True i plik istnieje, nowe dane będą dodawane na końcu istniejącego pliku
    def __init__(self, file_path, append_mode=False):

        # write_pos: Aktualna pozycja zapisu w buforze
        self.write_pos = 0

        # size: Rozmiar bufora, ustalony wcześniej jako BUFFER_SIZE
        self.size = BUFFER_SIZE

        # buffer: Lista, która będzie przechowywać zapisywane rekordy.
        # początkowo wypełniona wartościami None
        self.buffer = [None] * BUFFER_SIZE

        # file_path: Ścieżka do pliku, do którego będą zapisywane dane
        self.file_path = file_path

        # usuwam istniejący plik, jeśli append_mode jest ustawione na False i plik już istnieje
        if not append_mode and os.path.isfile(file_path):
            os.remove(file_path)

        # runs_written: Liczba wykonanych zapisów (run) do pliku
        self.runs_written = 0

        # last_written: Ostatni zapisany rekord
        self.last_written = None

        # disk_writes_count: Licznik operacji zapisu na dysku
        self.disk_writes_count = 0

    # write_next(self, record): Metoda służąca do zapisu kolejnego rekordu do bufora
    # record: Rekord do zapisania
    def write_next(self, record):

        # Sprawdzenie, czy aktualnie zapisywany rekord (record) jest mniejszy od ostatnio zapisanego (self.last_written).
        # Jeśli tak, zwiększa licznik runs_written
        if record < self.last_written:
            self.runs_written += 1

        # Sprawdzenie, czy bufor (self.buffer) jest pełny (self.write_pos == self.size).
        # Jeśli tak, wywołuje metodę flush(), która zapisuje zawartość bufora do pliku.
        if self.write_pos == self.size:
            self.flush()

        # zapisuje rekord do bufora na aktualnej pozycji self.write_pos
        self.buffer[self.write_pos] = record
        self.write_pos += 1

        # ustawiam self.last_written na aktualnie zapisany rekord (record)
        self.last_written = record


    # save_next(self): Metoda zapisująca zawartość bufora do pliku
    def save_next(self):

        # tworzę listę ints_to_write, która zawiera zserializowane wartości rekordów znajdujących się w buforze.
        ints_to_write = []


        for record in self.buffer[0:self.write_pos]:
            ints_to_write += record.save_to_ints()  # type: ignore

        # otwieram plik w trybie do dopisywania binarnego ("ab"), bez buforowania (buffering=0)
        file = open(self.file_path, "ab", buffering=0)

        # konwertuje listę ints_to_write na tablicę bajtów (bytearray) i zapisuje ją do pliku
        file.write(bytearray(ints_to_write))

        file.close()

        # zwiększam licznik operacji zapisu na dysku (self.disk_writes_count)
        self.disk_writes_count += 1

    # flush(self): Metoda zapisująca zawartość aktualnie wypełnionego bufora do pliku
    def flush(self):

        # sprawdzam, czy self.write_pos (aktualna pozycja zapisu w buforze) jest większa od 0.
        # Jeśli tak, wywołuje metodę save_next() do zapisania danych
        if self.write_pos > 0:
            self.save_next()

            # resetuje self.write_pos do 0, przygotowując bufor do dalszych operacji zapisu
            self.write_pos = 0

In [56]:
import pdb
class Record:


    # konstruktor inicjalizuje obiekt Record z listą elementów (items)
    def __init__(self, items):
        self.items = items

    # Metoda ta służy do tworzenia obiektu Record z listy liczb całkowitych.


    @staticmethod
    def load_from_ints(record_ints):
        #pdb.set_trace()
        # #Pierwsza liczba (set_length) określa długość zestawu
        set_length = record_ints[0]
        # kolejne liczby to elementy tego zestawu
        set_items = record_ints[1:set_length + 1]

        # tworzę nowy obiekt Record z tymi elementami
        return Record(set_items)



    def save_to_ints(self):

        # wynikowa lista zawiera długość zestawu jako pierwszy element, następnie elementy zestawu
        result = [len(self.items), *self.items]

        # resztę wypełniają zera, aby osiągnąć rozmiar RECORD_BYTES_SIZE
        zeropad = [0] * (RECORD_BYTES_SIZE - len(result))
        result += zeropad
        return result

    # metoda __repr__(self) - zwraca czytelną reprezentację obiektu Record jako posortowaną listę elementów w odwrotnej kolejności (reverse=True) .
    def __repr__(self):
        return f"Set {sorted(self.items, reverse=True)}"



    # Metoda __lt__ w klasie Record implementuje operator porównania "mniejszy niż" (<) dla obiektów tej klasy.
    # Dzięki tej metodzie można porównywać dwa obiekty Record i określić, czy jeden jest mniejszy od drugiego zgodnie z określonymi kryteriami.
    def __lt__(self, other):
        # Jeśli drugi obiekt (other) jest None, obecny obiekt (self) jest uznawany za mniejszy
        if other is None:
            return True
        #pdb.set_trace()
        self_items_copy = self.items[:]
        other_items_copy = other.items[:]

        # Jeśli element z tej listy znajduje się również w kopii listy elementów drugiego obiektu (other_items_copy), to ten element jest usuwany z obu list
        for item in self_items_copy:
            if item in other_items_copy:
                self_items_copy.remove(item)
                other_items_copy.remove(item)


        # Jeśli po usunięciu wspólnych elementów lista elementów drugiego obiektu (other_items_copy) jest pusta, oznacza to,
        # że wszystkie jego elementy były wspólne z obecnym obiektem, więc obecny obiekt (self) nie jest mniejszy od drugiego obiektu (False).
        if len(other_items_copy) == 0:
            return False

        # Jeśli lista elementów obecnego obiektu (self_items_copy) jest pusta, oznacza to,
        # że wszystkie jego elementy były wspólne z drugim obiektem, więc obecny obiekt (self) jest mniejszy od drugiego obiektu (True)
        elif len(self_items_copy) == 0:
            return True

        # Jeśli obie listy zawierają jeszcze jakieś elementy, metoda porównuje największe pozostałe elementy z obu list
        # (s_max dla obecnego obiektu i o_max dla drugiego obiektu).
        s_max = max(self_items_copy)
        o_max = max(other_items_copy)

        # # Obecny obiekt jest mniejszy, jeśli największy element z drugiego obiektu (o_max) jest większy od największego elementu z obecnego obiektu (s_max)
        return o_max > s_max

In [58]:
# Klasa RunIterator działa jako iterator umożliwiający iterację po kolejnych rekordach z obiektu ReadBuffer.

#Jest zoptymalizowana pod kątem pracy z dużymi zbiorami danych, zapewniając efektywne odczytywanie i monitorowanie przebiegów rekordów.

# Flagę end_of_run wykorzystuje się do wykrywania końca bieżącego przebiegu rekordów,
# co jest istotne w algorytmach sortowania, takich jak naturalne scalanie.

# Dzięki implementacji interfejsu iteratora (__iter__ i __next__), klasa RunIterator może być używana w konstrukcjach iteracyjnych Pythona,
#co znacząco ułatwia przetwarzanie i analizę danych

class RunIterator:

    # Konstruktor inicjuje nową instancję RunIterator
    def __init__(self, read_buffer):

        # read_buffer: Obiekt ReadBuffer, który zawiera dane do iteracj
        self.read_buffer = read_buffer

        # current_record: Aktualny rekord, który jest aktualnie przetwarzany
        self.current_record = None

        # end_of_run: Flaga określająca, czy aktualny przebieg (run) rekordów dobiegł końca
        self.end_of_run = False


    # read_next(self): Metoda służy do odczytywania kolejnych rekordów z read_buffer
    def read_next(self):

        # sprawdzam, czy flaga end_of_run jest ustawiona na True.
        # Jeśli tak, zwraca None, co oznacza, że nie ma więcej rekordów do odczytu
        if self.end_of_run:
            return None

        # Odczytuje następny rekord z read_buffer i przypisuje go do self.current_record
        self.current_record = self.read_buffer.read_next()

        # Jeśli self.current_record jest None, również zwraca None, co sygnalizuje koniec danych do odczytu
        if self.current_record is None:
            return None

        # Korzystając z metody peek() z read_buffer, sprawdzam następny rekord bez przesuwania wskaźnika odczytu.
        next_record = self.read_buffer.peek()

        # Jeśli istnieje i jest mniejszy niż self.current_record, ustawia end_of_run na True,
        # co oznacza zakończenie bieżącego przebiegu rekordów
        if next_record is not None and next_record < self.current_record:
            self.end_of_run = True

        return self.current_record


    def __iter__(self):
        return self

    # __next__(self): Metoda zwracająca następny rekord w iteracji
    def __next__(self):

        # Wywołuje metodę read_next() do odczytania kolejnego rekordu
        res_record = self.read_next()

        # Jeśli res_record (rezultat odczytania) jest None, podnosi wyjątek StopIteration, co sygnalizuje zakończenie iteracji
        if res_record is None:
            raise StopIteration
        return res_record

In [60]:
# Funkcja print_tape(file_name) umożliwia analizę zawartości taśmy (pliku) poprzez
# wypisanie każdego rekordu oraz podsumowanie liczby serii i rekordów.

# Jest to przydatne narzędzie do weryfikacji poprawności danych, przetwarzania wstępnego przed dalszymi operacjami,
# takimi jak sortowanie, analiza lub transformacja danych.

# Dzięki iteracyjnemu podejściu za pomocą RunIterator funkcja może obsługiwać nawet duże zbiory danych efektywnie,
# minimalizując zużycie pamięci i zapewniając możliwość stopniowego przetwarzania danych


def print_tape(file_name):

    # wyświetlam nagłówek, który informuje o nazwie taśmy (pliku), którą będe przetwarzać.
    print(f"[ .. ] Tape {file_name}\n")

    # tworzy instancję ReadBuffer dla podanego pliku file_name, który zawiera dane
    buffer = ReadBuffer(file_name)

    # series_count - inicjacja liczby serii
    series_count = 0

    # records_count inicjacja liczby rekordów
    records_count = 0


    # while buffer.has_more(): sprawdza, czy w buforze są jeszcze dane do odczytu
    while buffer.has_more():

        # ri = RunIterator(buffer) tworzy iterator RunIterator na podstawie buffer,
        # co umożliwia iteracyjne odczytywanie kolejnych rekordów z taśmy
        ri = RunIterator(buffer)

        # for record in ri: iteruje po rekordach zwróconych przez RunIterator
        for record in ri:

            # każdy rekord jest wyświetlany za pomocą print(record)
            print(record)

            # licznik records_count zwiększa się o jeden po każdym wyświetleniu rekordu
            records_count += 1

        series_count += 1

        # po przejściu przez wszystkie rekordy w bieżącej serii,
        # wyświetlam print("~ series end ~") oznaczające koniec serii rekordów
        print("~ series end ~")

    # po zakończeniu iteracji po wszystkich serii, wyświetlam podsumowanie

    # Series count: liczba serii (grup rekordów) na taśmie
    print(f"\n[ ^- ] Series count: {series_count}")

    # Records count: całkowita liczba rekordów na taśmie
    print(f"[ ^- ] Records count: {records_count}")

In [62]:
# Funkcja print_runs(file_name, n) jest użyteczna do weryfikacji zawartości i
# struktury danych na taśmie (pliku tekstowym) poprzez wypisanie pierwszych n serii.

# Umożliwia to szybkie zrozumienie, jak dane są zorganizowane i czy są zgodne z oczekiwaniami.
# Dzięki użyciu ReadBuffer i RunIterator, funkcja może efektywnie zarządzać dużymi plikami danych,
# minimalizując obciążenie pamięciowe poprzez iteracyjne przetwarzanie


def print_runs(file_name, n):

    # wyświetlam informację o tym, że zostaną wydrukowane pierwsze n serii danych z pliku file_name
    print(f"Printing first {n} runs from {file_name}")

    # buff = ReadBuffer(file_name): Tworzy instancję ReadBuffer dla podanego pliku file_name,
    # co umożliwia odczyt danych z taśm
    buff = ReadBuffer(file_name)

    # for i in range(n): Pętla iteruje n razy, co odpowiada ilości serii (run), które chcemy wydrukować.
    for i in range(n):

        # print(f"\nRun {i}:"): wyświetlam nagłówek informujący o numerze aktualnej serii.
        print(f"\nRun {i}:")

        # ri = RunIterator(buff): Tworzy iterator RunIterator na podstawie bufora buff.
        # RunIterator pozwala na odczyt kolejnych rekordów z taśmy
        ri = RunIterator(buff)

        # for record in ri: Pętla iteruje po rekordach zwróconych przez RunIterator
        for record in ri:
            # Każdy rekord jest wyświetlany za pomocą print(record),
            # co umożliwia wizualizację zawartości kolejnych serii danych
            print(record)

In [65]:
# Funkcja runs_count(file_name) jest użyteczna do określenia liczby serii (run)
# danych znajdujących się w pliku tekstowym.

# Jest to przydatne w kontekście analizy i zarządzania dużymi zbiorami danych,
# gdzie istotne jest zrozumienie ich struktury i organizacji.

# Dzięki wykorzystaniu ReadBuffer i RunIterator, funkcja efektywnie zarządza odczytem danych z pliku,
# co jest szczególnie przydatne przy pracy z dużymi plikami, gdzie nie można założyć,
#że cały plik będzie przechowywany w pamięci operacyjnej na raz

def runs_count(file_name):

    # Zmienna rc inicjalizowana jest na początku jako zero.
    # Będzie ona przechowywać liczbę serii (run) danych.
    rc = 0

    # buff = ReadBuffer(file_name): Tworze instancję ReadBuffer dla podanego pliku file_name,
    # co umożliwia odczyt danych z taśmy
    buff = ReadBuffer(file_name)

    # while buff.has_more():: Pętla wykonuje się dopóki są dostępne kolejne rekordy do odczytu z bufora buff
    while buff.has_more():

        # ri = RunIterator(buff): Tworzy iterator RunIterator na podstawie bufora buff.
        # RunIterator pozwala na odczyt kolejnych rekordów z taśmy.
        ri = RunIterator(buff)

        # for _ in ri: Pętla iteruje po rekordach zwróconych przez RunIterator,
        # ale nie wykonuje żadnych operacji wewnętrznych (pass oznacza pominięcie).
        for _ in ri:
            pass

        # Po przeiterowaniu wszystkich rekordów w aktualnej serii,
        # inkrementuje się zmienną rc o jeden, co zwiększa liczbę serii.
        rc += 1

    # Po zakończeniu iteracji po wszystkich serii,
    # funkcja zwraca całkowitą liczbę serii (run), które zostały policzone
    return rc

In [66]:
# Funkcja prepare_tapes() ma na celu przygotowanie początkowego stanu taśm t1, t2 i t3
# przed wykonaniem operacji sortowania lub innego przetwarzania danych na tych taśmach.

# Dzięki temu:

#Taśma t1 jest zapełniana danymi z taśmy startowej start_tape

# Taśmy t2 i t3 są czyszczone, aby usunąć ewentualne wcześniejsze dane,
# co zapobiega konfliktom lub problemom podczas dalszych operacji.

# To wszystko sprawia, że funkcja jest ważnym elementem w procesie przygotowywania
# danych do dalszej obróbki, zapewniając, że wszystkie taśmy są gotowe i czyste
# przed rozpoczęciem głównego algorytmu sortowania lub przetwarzania danych.


def prepare_tapes():

    # tworze obiekt WriteBuffer o nazwie t1_dest, który będzie służył do zapisu danych do taśmy t1
    t1_dest = WriteBuffer("/content/t1")

    # otwieram bufor do odczytu danych z pliku start_tape za pomocą ReadBuffer

    # Operacja ta kontynuuje się, dopóki istnieją kolejne rekordy do odczytu z taśmy startowej
    for record in ReadBuffer("/content/start_tape"):

        # Każdy rekord odczytany z ReadBuffer jest następnie zapisywany
        # do WriteBuffer (t1_dest) za pomocą metody write_next(record)
        t1_dest.write_next(record)

    # Po zapisaniu wszystkich rekordów do WriteBuffer (t1_dest),
    # wywoływołuje metodę flush(), która zapisuje zawartość bufora do pliku t1
    t1_dest.flush()

    # Sprawdzam, czy istnieją pliki t2 i t3

    # Operacja ta zapewnia, że przed dalszymi operacjami na taśmach t2 i t3,
    # ich zawartość jest wyczyszczona lub zresetowana
    if os.path.isfile("/content/t2"):
        os.remove("/content/t2")
    if os.path.isfile("/content/t3"):
        os.remove("/content/t3")